In [1]:
import pandas as pd
from pandas_profiling import ProfileReport

# Load the dataset
df = pd.read_csv("melb_data.csv")


<ipython-input-1-d167e8a4bf93>:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [7]:
# Task 1: Handling Missing Values
missing_percent = df.isna().sum() / len(df) * 100
columns_to_drop = missing_percent[missing_percent > 20].index
df.drop(columns=columns_to_drop, inplace=True)

# Fill missing numerical values with median
df.fillna(df.select_dtypes(include=['number']).median(), inplace=True)

# Fill missing categorical values with mode
for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

<ipython-input-7-e911adcbbb21>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [8]:
# Task 2: Filtering and Indexing
if "Suburb" in df.columns and "Price" in df.columns:
    richmond_houses = df[(df["Suburb"] == "Richmond") & (df["Price"] > 1000000)]
else:
    richmond_houses = pd.DataFrame()

if "Landsize" in df.columns and "Price" in df.columns and "Suburb" in df.columns and "BuildingArea" in df.columns:
    df_filtered = df.loc[df["Landsize"] > 500, ["Price", "Suburb", "BuildingArea"]]
else:
    df_filtered = pd.DataFrame()

if "Price" in df.columns:
    df_sorted = df.sort_values(by="Price", ascending=False)
else:
    df_sorted = pd.DataFrame()

In [9]:
# Task 3: Data Transformation & Feature Engineering
if "Price" in df.columns and "Rooms" in df.columns:
    df["Price_per_Room"] = df["Price"] / df["Rooms"]

if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    df.dropna(subset=["Date"], inplace=True)
    df["Year_Sold"] = df["Date"].dt.year

In [10]:
# Task 4: Aggregation and Grouping
suburb_avg_price = df.groupby("Suburb")["Price"].mean()
properties_per_suburb = df.groupby("Suburb")["Price"].count()
most_expensive_suburb = suburb_avg_price.idxmax()

In [2]:
# Task 5: Exporting Data
df.to_csv("cleaned_melb_data.csv", index=False)
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_file("pandas_profiling_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
# Bonus Challenge
median_price = df["Price"].median()
df["Category"] = df["Price"].apply(lambda x: "Expensive" if x > median_price else "Affordable")
correlation = df[["Price", "Rooms"]].corr()

In [11]:
# Save summary report
with open("summary_report.txt", "w") as f:
    f.write(f"Most expensive suburb: {most_expensive_suburb}\n")
    f.write(f"Price-Room Correlation:\n{correlation}\n")

